In [13]:
import cv2 as cv
import numpy as np
import os

### **Generalización para múltiples detecciones de un mismo template**

Abarca items 1, 2 y 3.

In [14]:
# --- Ruta de la carpeta con las imágenes ---
directorio = "./images"

# --- Lista para guardar las imágenes ---
imagenes = []

# --- Recorremos los archivos del directorio ---
for archivo in os.listdir(directorio):
    ruta_completa = os.path.join(directorio, archivo)

    # --- Verificamos que sea un archivo y termine en una extensión de imagen ---
    if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg')):
        img = cv.imread(ruta_completa)
        if img is not None:
            imagenes.append((archivo.split(".")[0], img))
        else:
            print(f"No se pudo leer la imagen: {archivo}")

print(f"Se leyeron {len(imagenes)} imágenes")

Se leyeron 7 imágenes


In [15]:
def preproc_img(img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_blur = cv.GaussianBlur(img_gray, (5,5), 0)
    img_edges = cv.Canny(img_blur, 50, 150)
    return img_edges

In [17]:
# --- Carpeta de salida ---
output_folder = "./resultados"
os.makedirs(output_folder, exist_ok=True)

# --- Cargar template ---
template = cv.imread("./template/pattern.png")
template_preproc = preproc_img(template)
h, w = template_preproc.shape

# --- Factores de escala a probar ---
scales = np.round(np.linspace(0.3, 4.5, 40), 2)  # 50% a 150%

for nombre, img in imagenes:
    best_val = -1
    best_scale = 1.0
    best_result = None
    best_resized = None
    max_roi = None

    # Buscar mejor escala
    for scale in scales:
        i = img.copy()
        resized = cv.resize(i, None, fx=scale, fy=scale)

        if resized.shape[0] < h or resized.shape[1] < w:
            continue

        img_preproc = preproc_img(resized)

        if img_preproc is None:
            continue

        result = cv.matchTemplate(img_preproc, template_preproc, cv.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        # ROI de la coincidencia
        top_left = max_loc
        roi = img_preproc[top_left[1]:top_left[1]+h, top_left[0]:top_left[0]+w]

        # Aplicamos una doble validación para eliminar flasos positivos usando ORB
        # Nos quedamos con 100 features del template y de la ROI y consideramos
        # true positives si hay un mínimo de 25 features coincidentes
        orb = cv.ORB_create(nfeatures=100)
        kp1, des1 = orb.detectAndCompute(template_preproc, None)
        kp2, des2 = orb.detectAndCompute(roi, None)

        # Evaluar si se encontraron descriptores en el template y en la ROI
        if des1 is None or des2 is None:
            continue

        bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)

        if len(matches) < 25:  # umbral mínimo de coincidencias
            continue

        if max_val > best_val:
            best_val = max_val
            best_scale = scale
            best_result = result
            best_resized = resized.copy()
            max_roi = roi.copy()

    # Obtener TODAS las detecciones de la mejor escala
    if best_result is not None:
        threshold = best_val * 0.7
        loc = np.where(best_result >= threshold)
        rects = []

        for pt in zip(*loc[::-1]):
            rects.append([pt[0], pt[1], w, h])

        # Agrupar rectángulos solapados
        rects, weights = cv.groupRectangles(rects, groupThreshold=1, eps=0.5)

        # Dibujar rectángulos
        for (x, y, w, h) in rects:
            cv.rectangle(best_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Agregar métricas sobre la imagen
        text = f"Escala: {best_scale:.2f} | MaxVal: {best_val:.4f} | Detecciones: {len(rects)}"
        print(text)

        # Guardar imagen
        output_path = os.path.join(output_folder, f"{nombre}_result.png")
        cv.imwrite(output_path, best_resized)

        if max_roi is not None:
            output_path = os.path.join(output_folder, f"{nombre}_roi.png")
            cv.imwrite(output_path, max_roi)

        print(f"Imagen {nombre} guardada con detecciones en {os.path.join(output_folder, f'{nombre}_result.png')}")
    else:
        print("No se encontró coincidencia.")

Escala: 0.30 | MaxVal: 0.0665 | Detecciones: 1
Imagen COCA-COLA-LOGO guardada con detecciones en ./resultados/COCA-COLA-LOGO_result.png
Escala: 4.07 | MaxVal: 0.1137 | Detecciones: 14
Imagen coca_multi guardada con detecciones en ./resultados/coca_multi_result.png
Escala: 2.35 | MaxVal: 0.2232 | Detecciones: 1
Imagen coca_logo_1 guardada con detecciones en ./resultados/coca_logo_1_result.png
Escala: 0.73 | MaxVal: 0.0880 | Detecciones: 1
Imagen coca_retro_1 guardada con detecciones en ./resultados/coca_retro_1_result.png
Escala: 1.27 | MaxVal: 0.0990 | Detecciones: 1
Imagen logo_1 guardada con detecciones en ./resultados/logo_1_result.png
Escala: 1.81 | MaxVal: 0.0690 | Detecciones: 1
Imagen coca_logo_2 guardada con detecciones en ./resultados/coca_logo_2_result.png
Escala: 2.45 | MaxVal: 0.1629 | Detecciones: 1
Imagen coca_retro_2 guardada con detecciones en ./resultados/coca_retro_2_result.png


### **Conclusiones**

Se realizó un resize de las imágenes objetivos entre las escalas 0.3 y 4.5 con 40 pasos entre ellas.

Se hizo un preprocesamiento tanto del template como las imágenes usando primero conversión a escala de grises, desenfoque Gaussiano y detección de bordes mediante método de Canny.

Se aplicó matching template sobre las escalas, quedándose con la ROI de cada imagen (preprocesada) con mejor correlación. Luego se aplicó una doble validación para eliminar flasos positivos usando ORB entre la ROI y el template. Nos quedamos con 100 features del template y de la ROI, y consideramos true positives si hay un mínimo de 25 features coincidentes.

Además, para poder hacer múltiples detecciones se tomo el valor de correlación de la ROI y se utilizó como umbral un 70% de este para mostrar el resto de detecciones.

Finalmente se unificó el código para agrupar todos recuadros de los matches de manera de no tener superposiciones y se mostró la mejor correlación obtenida.